In [ ]:
# pip install langchain qdrant_client openai streamlit

In [ ]:
### DO NOT RUN ALL ###

In [ ]:
### these are functions to run individually as needed to modify the Qdrant collections!! the py. files are to run the script ###

In [1]:
from dotenv import load_dotenv
import streamlit as st

In [2]:
# Imports
from langchain.vectorstores import Qdrant
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
from qdrant_client import QdrantClient, models
import os
import qdrant_client


In [3]:
# create qdrant client
client = qdrant_client.QdrantClient(
        st.secrets["QDRANT_HOST"], 
        api_key=st.secrets["QDRANT_API_KEY"]
    )

In [5]:
# delete collection

client.delete_collection(collection_name=st.secrets["QDRANT_COLLECTION_NAME"])

True

In [6]:
# create collection

# define parameters for vectors
vectors_config = models.VectorParams(
    size=1536, # vector size for OpenAI
    distance=models.Distance.COSINE
)

# create collection
client.recreate_collection(
    collection_name=st.secrets["QDRANT_COLLECTION_NAME"],
    vectors_config=vectors_config,
)

True

In [7]:
# create vector store

embeddings = OpenAIEmbeddings()

# get collection name
vector_store = Qdrant(
    client=client, 
    collection_name=st.secrets("QDRANT_COLLECTION_NAME"), 
    embeddings=embeddings,
)

c:\Users\derek\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [ ]:
# # add 'extracted text' to vector store

# from langchain_text_splitters import RecursiveCharacterTextSplitter

# def get_text_chunks(text):
    
#     text_splitter = RecursiveCharacterTextSplitter(
#         chunk_size=1000,
#         chunk_overlap=200,
#         length_function=len,
#         is_separator_regex=False,
#     )
     
#     chunks = text_splitter.split_text(text)
#     return chunks

# with open('combined_markdown_and_code.txt') as f:
#     raw_text = f.read()
    
# texts = get_text_chunks(raw_text)

# metadatas = [{"name": raw_text} for _ in texts]
# vector_store.add_texts(texts, metadatas=metadatas)

In [8]:
from langchain_community.document_loaders import PyPDFLoader
# add 'extracted pdf' to vector store

from langchain_text_splitters import RecursiveCharacterTextSplitter

def get_pdf_chunks(pdf_filepath):
    loader = PyPDFLoader(pdf_filepath)
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200,
    )
     
    chunks = text_splitter.split_text(pdf_filepath)
    return chunks


slides_filepath = 'Slides'
for slide in os.listdir(slides_filepath):
    slide_path = os.path.join(slides_filepath, slide)
    texts = get_pdf_chunks(slide_path)
    metadatas = [{"name": slide} for _ in texts]
    vector_store.add_texts(texts, metadatas=metadatas)
    


In [9]:
# add transcipts to vector store

from langchain_text_splitters import RecursiveCharacterTextSplitter

def get_text_chunks(text_filepath):
    
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len,
        is_separator_regex=False,
    )
    chunks = text_splitter.split_text(text_filepath)
    return chunks


transcripts_filepath = 'Transcripts'
for transcript in os.listdir(transcripts_filepath):
    texts = get_text_chunks(transcript)
    metadatas = [{"name": transcript} for _ in texts]
    vector_store.add_texts(texts, metadatas=metadatas)
    


In [10]:
# add output folder containing Activities to vector store

from langchain_text_splitters import RecursiveCharacterTextSplitter

def get_text_chunks(text_filepath):
    
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len,
        is_separator_regex=False,
    )
    chunks = text_splitter.split_text(text_filepath)
    return chunks


transcripts_filepath = 'Output'
for transcript in os.listdir(transcripts_filepath):
    texts = get_text_chunks(transcript)
    metadatas = [{"name": transcript} for _ in texts]
    vector_store.add_texts(texts, metadatas=metadatas)